In [3]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
%matplotlib inline 

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.3f}'.format

In [ ]:
market_swap_txs_v1 =  pd.read_csv('./txs_data/v1/market_swap_txs.csv')
market_swap_send_txs_v1 =  pd.read_csv('./txs_data/v1/market_swap_send_txs.csv')

# market_swap_txs_v2 =  pd.read_csv('./txs_data/v2/market_swap_txs.csv')
# market_swap_send_txs_v2 =  pd.read_csv('./txs_data/v2/market_swap_send_txs.csv')

# market_swap_txs_v3 =  pd.read_csv('./txs_data/v3/market_swap_txs.csv')
# market_swap_send_txs_v3 =  pd.read_csv('./txs_data/v3/market_swap_send_txs.csv')

# market_swap_txs_v4 =  pd.read_csv('./txs_data/v4/market_swap_txs.csv')
# market_swap_send_txs_v4 =  pd.read_csv('./txs_data/v4/market_swap_send_txs.csv')

# market_swap_txs_v5 =  pd.read_csv('./txs_data/v5/market_swap_txs.csv')
# market_swap_send_txs_v5 =  pd.read_csv('./txs_data/v5/market_swap_send_txs.csv')

# market_swap_txs_v6 =  pd.read_csv('./txs_data/v6/market_swap_txs.csv')
# market_swap_send_txs_v6 =  pd.read_csv('./txs_data/v6/market_swap_send_txs.csv')

# market_swap_txs_v7 =  pd.read_csv('./txs_data/v7/market_swap_txs.csv')
# market_swap_send_txs_v7 =  pd.read_csv('./txs_data/v7/market_swap_send_txs.csv')


exchange_rate_vote_txs_v1 =  pd.read_csv('./txs_data/v1/exchange_rate_vote_txs.csv')
# exchange_rate_vote_txs_v2 =  pd.read_csv('./txs_data/v2/exchange_rate_vote_txs.csv')
# exchange_rate_vote_txs_v3 =  pd.read_csv('./txs_data/v3/exchange_rate_vote_txs.csv')
# exchange_rate_vote_txs_v4 =  pd.read_csv('./txs_data/v4/exchange_rate_vote_txs.csv')
# exchange_rate_vote_txs_v5 =  pd.read_csv('./txs_data/v5/exchange_rate_vote_txs.csv')
# exchange_rate_vote_txs_v6 =  pd.read_csv('./txs_data/v6/exchange_rate_vote_txs.csv')
# exchange_rate_vote_txs_v7 =  pd.read_csv('./txs_data/v7/exchange_rate_vote_txs.csv')

astroport_ust_luna_txs_v1 =  pd.read_csv('./astroport_txs_data/astroport_swap_txs.csv')


In [ ]:
astroport_ust_luna_txs_v1.head(4)

In [5]:
astroport_ust_luna_txs_v1.head(4)

,TxHash,BlockHeight,UserAddress,offer_asset,offer_amount,ask_asset,return_amount,tax_amount,spread_amount,commission_amount,maker_fee_amount,belief_price
0,06560117D2FC9C7029084E53A7FA266E736CCDEF32861D9A5265709CEA5CF988,7622693,terra173cavyl684vfy4al20r3pzldy0jdljq0ma7vzc,uusd,800000,uluna,314302982,0,121,945746,315217,0.003
1,3DAD7540BB34F419E79609F6FF9941D3B6BAEDA80D966CC8C86CD21B38625D33,7622693,terra1j8mqyr56hlja2yms5umv5mdqmkj047ynzldjer,uluna,408218305180,uusd,1033334691,0,518800,3109332,1036340,397.100
2,DA6932436A65136A4F2DB3EDC55431A75BB15AEA0339C35C20BF10CBBBE24435,7622692,terra1yf7t6ju6jcxs489g69ceplcrz8lv2245vmzz9a,uusd,400000000,uluna,156993571306,0,30407327,472397907,157450222,0.003
3,AC87CF4BD4F151192D0BACDA12A22EC50612253AB593A4770A9F8C99F11BCC54,7622691,terra1sp4t8y4hr324a6fpeyscheet09mkkzwzq0sha5,uusd,1000000000,uluna,392748942895,0,190265970,1181792205,393891341,0.003


In [ ]:
market_swap_txs_DF = market_swap_txs_v1
market_swap_send_txs_DF = market_swap_send_txs_v1
exchange_rate_vote_txs_DF = exchange_rate_vote_txs_v1


# market_swap_txs_DF = pd.concat([market_swap_txs_v1, market_swap_txs_v2, market_swap_txs_v3, market_swap_txs_v4, market_swap_txs_v5, market_swap_txs_v6, market_swap_txs_v7])
# market_swap_send_txs_DF = pd.concat([market_swap_send_txs_v1, market_swap_send_txs_v2, market_swap_send_txs_v3, market_swap_send_txs_v4, market_swap_send_txs_v5, market_swap_send_txs_v6, market_swap_send_txs_v7])
# exchange_rate_vote_txs_DF = pd.concat([exchange_rate_vote_txs_v1, exchange_rate_vote_txs_v2, exchange_rate_vote_txs_v3, exchange_rate_vote_txs_v4, exchange_rate_vote_txs_v5, exchange_rate_vote_txs_v6, exchange_rate_vote_txs_v7])


In [ ]:
market_swap_txs_DF.sort_values('BlockHeight', inplace=True)
market_swap_send_txs_DF.sort_values('BlockHeight', inplace=True)
exchange_rate_vote_txs_DF.sort_values('BlockHeight', inplace=True)

In [ ]:
exchange_rate_vote_txs_DF = exchange_rate_vote_txs_DF[:1000]

In [ ]:
# Reset Indexes
exchange_rate_vote_txs_DF = exchange_rate_vote_txs_DF.reset_index()
market_swap_send_txs_DF = market_swap_send_txs_DF.reset_index()
market_swap_txs_DF = market_swap_txs_DF.reset_index()



In [ ]:
exchange_rate_vote_txs_DF.head(4)

In [ ]:
len(exchange_rate_vote_txs_DF.index)

In [ ]:
def index_exchange_rates(column,denom):
    words = column.split(",")
    for word in words:
        val = word[:len(word)-4]
        label = word[len(word)-4:]
        if label == denom:
            return val
    return 0

def get_labels(column):
    words = column.split(",")
    labels = []
    for word in words:
        val = word[:len(word)-4]
        label = word[len(word)-4:]
        labels.append(label)
    return labels



In [ ]:
# list of all currencies
labels = get_labels( exchange_rate_vote_txs_DF.loc[0]["exchange_rates"]  )

# Index Exchange rate for all currencies
for label in labels:
    print(f"Indexing for Label = {label}")
    exchange_rate_vote_txs_DF[label] = exchange_rate_vote_txs_DF.apply(lambda x: index_exchange_rates(x["exchange_rates"],label), axis=1)




In [ ]:
exchange_rate_vote_txs_DF.drop(['index','TxHash','salt','feeder','validator','exchange_rates'],axis=1,inplace=True)


In [ ]:
exchange_rate_vote_txs_DF.head(4)

In [ ]:
market_swap_txs_DF.head(4)

In [ ]:
market_swap_txs_DF = market_swap_txs_DF.fillna(0)
market_swap_txs_DF.drop(['index','TxHash','Sender','burner','minter','trader','recipient'],axis=1,inplace=True)


